In [1]:
import numpy as np
import pandas as pd
import magec_utils as mg
import mimic_utils as mimic
import matplotlib
import matplotlib.pyplot as plt
from sklearn.utils import class_weight

pd.set_option('display.max_columns', None)

%matplotlib inline

Using TensorFlow backend.


In [2]:
# MIMIC-III
vitals = ['heartrate_mean', 'sysbp_mean', 'diasbp_mean', 'meanbp_mean',
          'resprate_mean', 'tempc_mean', 'spo2_mean', 'glucose_mean']

labs = ['aniongap', 'albumin', 'bicarbonate', 'bilirubin', 'creatinine', 
        'chloride', 'glucose', 'hemoglobin', 'lactate', 
        'magnesium', 'phosphate', 'platelet', 'potassium', 'ptt', 'inr', 
        'pt', 'sodium', 'bun', 'wbc']  # -hematocrit

comobs = ['congestive_heart_failure', 'chronic_pulmonary', 'pulmonary_circulation']

others = ['age', 'gender']

features = vitals+labs

df_cohort = mimic.get_mimic_data()

In [3]:
df_ml = df_cohort.set_index(['subject_id', 'timepoint'])
df_ml = df_ml.sort_index(level=[0, 1], ascending=[1, 0])

In [4]:
def featurize(df, outcome):
    out = dict()
    for lab in labs:
        out[lab] = last_val(df[lab])
    for vital in vitals:
        out['first_'+vital] = first_val(df[vital])
        out['last_'+vital] = last_val(df[vital])
    for comob in comobs:
        out[comob] = last_val(df[comob])
    for other in others:
        out[other] = last_val(df[other])
    out['label'] = int(df[outcome].iloc[-1])
    return pd.Series(out)

def first_val(x):
    vals = list(x[~np.isnan(x)])
    if len(vals):
        return vals[0]
    else:
        return None

def last_val(x):
    vals = list(x[~np.isnan(x)])
    if len(vals):
        return vals[-1]
    else:
        return None

In [5]:
df_ml = df_ml.reset_index(1).groupby(level=0, group_keys=False).apply(lambda x: featurize(x, 'ventilated'))

In [6]:
from sklearn.preprocessing import StandardScaler

def impute(df):
    df[labs] = df[labs].fillna(df[labs].mean())
    for vital in vitals:
        df['first_'+vital] = df['first_'+vital].fillna(df['first_'+vital].mean())
        df['last_'+vital] = df['last_'+vital].fillna(df['last_'+vital].mean())    
    df[comobs] = df[comobs].fillna(0)
    return df

def train_valid_ml(df_ml, test_size=0.2, seed=7):
    np.random.seed(seed)

    x_cols = list(set(df_ml.columns) - {'subject_id', 'label'})
    y_cols = ['subject_id', 'label']

    cases = df_ml['subject_id'].unique()

    np.random.shuffle(cases)  # inplace shuffle

    valid_cases = cases[:int(len(cases) * test_size)]
    train_cases = cases[int(len(cases) * test_size):]

    train_cases = np.isin(df_ml['subject_id'], train_cases)
    valid_cases = np.isin(df_ml['subject_id'], valid_cases)

    xy_train = df_ml.loc[train_cases, :]
    x_train = xy_train[x_cols].copy()
    Y_train = xy_train[y_cols].copy()

    xy_valid = df_ml.loc[valid_cases, :]
    x_valid = xy_valid[x_cols].copy()
    Y_valid = xy_valid[y_cols].copy()

    x_train = impute(x_train)
    x_valid = impute(x_valid)

    stsc = StandardScaler()
    xst_train = stsc.fit_transform(x_train)
    xst_train = pd.DataFrame(xst_train, index=x_train.index, columns=x_train.columns)

    xst_valid = stsc.transform(x_valid)
    xst_valid = pd.DataFrame(xst_valid, index=x_valid.index, columns=x_valid.columns)

    return x_train, x_valid, stsc, xst_train, xst_valid, Y_train, Y_valid

In [7]:
x_train, x_valid, stsc, xst_train, xst_valid, Y_train, Y_valid = train_valid_ml(df_ml.reset_index())

In [8]:
def add_timepoint(df):
    df.index.names = ["case"]
    df = df.reset_index()
    df['timepoint'] = 0
    df = df.set_index(['case','timepoint'])
    return df

In [9]:
xst_valid = add_timepoint(xst_valid)
Y_valid = add_timepoint(Y_valid)

In [10]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn import svm

In [11]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced',
                                                  np.unique(Y_train['label']),
                                                  Y_train['label'])

In [12]:
class_weights

array([0.61911131, 2.59887711])

In [13]:
def create_mlp():
    mlp = Sequential()
    mlp.add(Dense(60, input_dim=len(xst_train.columns), activation='relu'))
    mlp.add(Dropout(0.2))
    mlp.add(Dense(30, input_dim=60, activation='relu'))
    mlp.add(Dropout(0.2))
    mlp.add(Dense(1, activation='sigmoid'))
    mlp.compile(loss='binary_crossentropy',
                loss_weights=[class_weights[0]], optimizer='adam', metrics=['accuracy'])
    return mlp

mlp = KerasClassifier(build_fn=create_mlp, epochs=100, batch_size=64, verbose=0)
mlp.fit(xst_train, Y_train['label'], epochs=100, batch_size=64, verbose=0)
print('Built MLP classifier!')

sv = svm.SVC(probability=True, class_weight="balanced")
sv.fit(xst_train, Y_train['label'])
print('Built SVM classifier!')

lr = LogisticRegression(C=1., class_weight='balanced', solver='lbfgs')
lr.fit(xst_train, Y_train['label'])
print('Built LR classifier!')

Built MLP classifier!
Built SVM classifier!
Built LR classifier!


In [14]:
rf = CalibratedClassifierCV(RandomForestClassifier(n_estimators=100,
                                                   min_samples_split=50,
                                                   min_samples_leaf=3,
                                                   max_features='sqrt',
                                                   max_depth=10,
                                                   bootstrap=True,
                                                   n_jobs=-1,
                                                   class_weight="balanced"),
                            method='sigmoid', cv=5)
rf.fit(xst_train, Y_train['label'])
print('Built RF classifier!')

Built RF classifier!


In [15]:
mg.evaluate(mlp, xst_valid, Y_valid['label'], verbose=True);

Accuracy: 0.860298
Precision: 0.769231
Recall: 0.431655
F1 score: 0.552995
ROC AUC: 0.824334
[[1612   54]
 [ 237  180]]


In [16]:
mg.evaluate(sv, xst_valid, Y_valid['label'], verbose=True);

Accuracy: 0.779165
Precision: 0.465154
Recall: 0.688249
F1 score: 0.555126
ROC AUC: 0.809303
[[1336  330]
 [ 130  287]]


In [17]:
mg.evaluate(lr, xst_valid, Y_valid['label'], verbose=True);

Accuracy: 0.672108
Precision: 0.338983
Recall: 0.671463
F1 score: 0.450523
ROC AUC: 0.721503
[[1120  546]
 [ 137  280]]


In [18]:
mg.evaluate(rf, xst_valid, Y_valid['label'], verbose=True);

Accuracy: 0.806049
Precision: 0.606557
Recall: 0.088729
F1 score: 0.154812
ROC AUC: 0.827203
[[1642   24]
 [ 380   37]]


In [19]:
magecs_lr = mg.case_magecs(lr, xst_valid, model_name='lr')
magecs_lr = mg.normalize_magecs(magecs_lr, features=None, model_name='lr')

magecs_svm = mg.case_magecs(sv, xst_valid, model_name='svm')
magecs_svm = mg.normalize_magecs(magecs_svm, features=None, model_name='svm')

magecs_mlp = mg.case_magecs(mlp, xst_valid, model_name='mlp')
magecs_mlp = mg.normalize_magecs(magecs_mlp, features=None, model_name='mlp')

In [20]:
vits = ['heartrate_mean', 'sysbp_mean', 'diasbp_mean', 
        'meanbp_mean', 'resprate_mean', 'spo2_mean']

drivers = ['first_' + x for x in vits] + ['last_' + x for x in vits]

In [21]:
feats = drivers + labs

In [22]:
joined = mg.magec_models(magecs_mlp, magecs_svm, magecs_lr, 
                         Xdata=xst_valid, Ydata=Y_valid['label'], features=feats)

In [23]:
def most_abnormal(x, features):
    res = None
    feat = None
    for f in features:
        if res is None or abs(x[f]) > res:
            res = abs(x[f])
            feat = f
    return feat

In [24]:
prob_cols = [c for c in joined.columns if c.startswith('perturb') and '_'.join(c.split('_')[1:-2]) in feats]

In [25]:
joined['orig_prob_ensemble'] = joined[['orig_prob_mlp', 'orig_prob_lr', 'orig_prob_svm']].apply(np.mean, 1)

In [26]:
joined[['best_feat', 'new_risk', 'rank_feat', 'rank_val', 'top_rank_prob']] = joined.apply(
    lambda x: mimic.best_feature(x, prob_cols), axis=1)

In [27]:
joined['most_abnormal'] = joined.apply(lambda x: most_abnormal(x, feats), axis=1)

In [31]:
joined[['orig_prob_ensemble', 'most_abnormal','best_feat', 'new_risk', 
        'rank_feat', 'rank_val', 'top_rank_prob']].head()

,orig_prob_ensemble,most_abnormal,best_feat,new_risk,rank_feat,rank_val,top_rank_prob
0,0.383190,last_resprate_mean,last_resprate_mean,0.278798,last_resprate_mean,-1.002250,0.278798
1,0.071798,platelet,first_diasbp_mean,0.044894,first_diasbp_mean,-0.577057,0.044894
2,0.274793,hemoglobin,aniongap,0.209148,aniongap,-0.770798,0.209148
3,0.224739,chloride,last_diasbp_mean,0.202640,last_sysbp_mean,-0.489628,0.204541
4,0.086288,hemoglobin,last_meanbp_mean,0.055838,last_meanbp_mean,-0.898212,0.055838


In [32]:
np.sum(joined['best_feat'] != joined['rank_feat']) / len(joined)

0.19635141622659624

In [33]:
def expected(x, drivers, sigmas=0.5, threshold=0.5):
    orig_prob_ensemble = x['orig_prob_ensemble']
    best_feat = x['best_feat']
    rank_feat = x['rank_feat']
    label = x['label']
    # some predicates
    cond1 = orig_prob_ensemble > threshold  # models predict MV (ventilated)
    cond2 = np.all(abs(x[drivers]) <= sigmas)  # all drivers are 'normal'
    cond3 = np.isin(best_feat, drivers) or np.isin(rank_feat, drivers) # MAgEC 'best/ranked feature' is a driver
#     cond4 = x[best_feat] > sigmas # MAgEC 'best feature' is 'abnormal'
    cond5 = label == 1  # patient was ventilated
    # Unexpected (ventilated):
    # 1. ensemble_probability greater than 0.5, 
    # 2. all drivers are normal
    # 3. best feature is not a driver 
    # 4. patient was ventillated
    if cond1 and cond2 and (not cond3) and cond5:
        return 'unexpected_ventilated_nondriver'
    elif cond1 and cond2 and cond3 and cond5:
        return 'unexpected_ventilated_driver'
    # Missed Unexpected (ventilated)
    elif (not cond1) and cond2 and cond5:
        return 'missed_unexpected_ventilated'
    # Expected (ventilated): 
    # 1. one or more drivers were abnormal
    # 2. patient was ventillated
    elif cond1 and (not cond2) and (not cond3) and cond5:
        return 'expected_ventilated_nondriver'
    elif cond1 and (not cond2) and cond3 and cond5:
        return 'expected_ventilated_driver'
    elif (not cond1) and (not cond2) and cond5:
        return 'missed_expected_ventilated'
    # Other (ventilated)
    elif cond5:
        return 'other_ventilated'
    # Unexpected (not ventilated)
    # 1. ensemble_probability less than 0.5
    # 2. one or more drivers are abnormal
    # 3. patient was not ventilated
    elif (not cond1) and (not cond2) and (not cond5):
        return 'unexpected_notventilated'
    # Expected (not ventilated)
    # 1. ensemble_probability less than 0.5
    # 2. all drivers are normal
    # 3. patient was not ventilated
    elif (not cond1) and cond2 and (not cond5):
        return 'expected_notventilated'
    elif cond1 and (not cond5):
        return 'missed_notventilated'
    elif (not cond5):
        return 'other_notventilated'
    else:
        return 'other'

In [34]:
joined['stats'] = joined.apply(lambda x: expected(x, drivers), axis=1)

In [35]:
joined['stats'].value_counts()

unexpected_notventilated           1564
missed_expected_ventilated          200
unexpected_ventilated_nondriver     124
missed_notventilated                 97
expected_ventilated_driver           75
missed_unexpected_ventilated          9
expected_ventilated_nondriver         8
expected_notventilated                5
unexpected_ventilated_driver          1
Name: stats, dtype: int64

In [36]:
excluded = set(df_ml[np.all(np.isnan(df_ml[drivers]), axis=1)].index.values)

In [37]:
len(excluded), df_cohort.subject_id.nunique()

(705, 10415)

In [38]:
filtered = joined[~np.isin(joined.case, list(excluded))]
len(joined), len(filtered)

(2083, 2047)

In [39]:
filtered['stats'].value_counts().sort_index()

expected_notventilated                5
expected_ventilated_driver           74
expected_ventilated_nondriver         8
missed_expected_ventilated          196
missed_notventilated                 96
missed_unexpected_ventilated          9
unexpected_notventilated           1535
unexpected_ventilated_driver          1
unexpected_ventilated_nondriver     123
Name: stats, dtype: int64

In [41]:
cols = drivers+['orig_prob_ensemble', 'new_risk', 'most_abnormal','best_feat', 
                'rank_feat', 'rank_val',  'top_rank_prob', 'stats','case']

filtered[filtered['stats'] == 'unexpected_ventilated_nondriver'][cols].head(10)

,first_heartrate_mean,first_sysbp_mean,first_diasbp_mean,first_meanbp_mean,first_resprate_mean,first_spo2_mean,last_heartrate_mean,last_sysbp_mean,last_diasbp_mean,last_meanbp_mean,last_resprate_mean,last_spo2_mean,orig_prob_ensemble,new_risk,most_abnormal,best_feat,rank_feat,rank_val,top_rank_prob,stats,case
13,-0.003458,-0.062489,-0.038729,-0.049032,0.01624,-0.009033,0.008451,-0.035547,-0.037171,-0.048262,0.050648,0.036792,0.642958,0.568236,lactate,aniongap,aniongap,-0.344505,0.568236,unexpected_ventilated_nondriver,53
32,-0.003458,-0.062489,-0.038729,-0.049032,0.01624,-0.009033,0.008451,-0.035547,-0.037171,-0.048262,0.050648,0.036792,0.749292,0.712471,sodium,aniongap,aniongap,-0.649668,0.712471,unexpected_ventilated_nondriver,140
33,-0.003458,-0.062489,-0.038729,-0.049032,0.01624,-0.009033,0.008451,-0.035547,-0.037171,-0.048262,0.050648,0.036792,0.624619,0.598686,albumin,bilirubin,albumin,-0.604841,0.599112,unexpected_ventilated_nondriver,147
44,-0.003458,-0.062489,-0.038729,-0.049032,0.01624,-0.009033,0.008451,-0.035547,-0.037171,-0.048262,0.050648,0.036792,0.701293,0.671838,albumin,wbc,wbc,-0.449433,0.671838,unexpected_ventilated_nondriver,187
79,-0.003458,-0.062489,-0.038729,-0.049032,0.01624,-0.009033,0.008451,-0.035547,-0.037171,-0.048262,0.050648,0.036792,0.760826,0.730942,hemoglobin,wbc,wbc,-0.809944,0.730942,unexpected_ventilated_nondriver,420
118,-0.003458,-0.062489,-0.038729,-0.049032,0.01624,-0.009033,0.008451,-0.035547,-0.037171,-0.048262,0.050648,0.036792,0.750825,0.697661,phosphate,phosphate,phosphate,-0.808123,0.697661,unexpected_ventilated_nondriver,639
120,-0.003458,-0.062489,-0.038729,-0.049032,0.01624,-0.009033,0.008451,-0.035547,-0.037171,-0.048262,0.050648,0.036792,0.600142,0.563376,glucose,pt,pt,-0.381452,0.563376,unexpected_ventilated_nondriver,652
123,-0.003458,-0.062489,-0.038729,-0.049032,0.01624,-0.009033,0.008451,-0.035547,-0.037171,-0.048262,0.050648,0.036792,0.582762,0.493781,hemoglobin,albumin,aniongap,-0.314123,0.526259,unexpected_ventilated_nondriver,663
136,-0.003458,-0.062489,-0.038729,-0.049032,0.01624,-0.009033,0.008451,-0.035547,-0.037171,-0.048262,0.050648,0.036792,0.718197,0.703782,albumin,phosphate,phosphate,-0.272070,0.703782,unexpected_ventilated_nondriver,721
140,-0.003458,-0.062489,-0.038729,-0.049032,0.01624,-0.009033,0.008451,-0.035547,-0.037171,-0.048262,0.050648,0.036792,0.695824,0.660301,hemoglobin,bilirubin,bilirubin,-0.658736,0.660301,unexpected_ventilated_nondriver,737


In [42]:
filtered[filtered['stats'] == 'unexpected_ventilated_nondriver']['rank_feat'].value_counts()

lactate        28
bilirubin      24
aniongap       23
wbc            12
albumin        12
hemoglobin      8
ptt             7
phosphate       6
pt              1
chloride        1
bicarbonate     1
Name: rank_feat, dtype: int64

In [43]:
filtered[filtered['stats'] == 'unexpected_ventilated_nondriver']['best_feat'].value_counts()

aniongap       32
lactate        29
bilirubin      17
ptt            12
wbc             9
hemoglobin      7
phosphate       7
albumin         4
bicarbonate     3
pt              1
inr             1
platelet        1
Name: best_feat, dtype: int64